# **Laboratorium 5** - Manipulacja danymi przy użyciu **Pandas**

## 4.1 Zepsucie datasetu

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import kagglehub as kg
from kagglehub import KaggleDatasetAdapter
import numpy as np

Przed rozpoczęciem poprawnego należy przygotować dane z typowymi błędami spotykanymi w datasetach.

In [ ]:
np.random.seed(42)
num_rows = 10000
clean_data = {
    'age': np.random.randint(18, 80, num_rows),
    'income': np.random.randint(20000, 150000, num_rows),
    'education': np.random.choice(['Podstawowa', 'Inżynier', 'Magister', 'Doktor', 'Doktor Habilitowany', 'Profesor'], num_rows),
    'city': np.random.choice(['Warszawa', 'Kraków', 'Gdańsk', 'Wrocław', 'Poznań', 'Bielsko-Biała', 'Katowice'], num_rows),
    'experience_years': np.random.randint(0, 40, num_rows),
    'school_satisfaction': np.random.randint(1, 11, num_rows),
    'department': np.random.choice(['IT', 'Ekonomia', 'Marketing', 'HR', 'Finanse', 'Logistyka', 'Filologia', 'Filozofia', 'Fizyka'], num_rows),
    'week_working_hours': np.random.randint(20, 60, num_rows),
    'commute_time': np.random.randint(5, 120, num_rows),
    'performance_score': np.random.uniform(1.0, 5.0, num_rows).round(2),
    'remote_days': np.random.randint(0, 6, num_rows)
}

df_clean = pd.DataFrame(clean_data)
df_clean

In [ ]:
import pandas as pd
import numpy as np
import random

def psuj_dataset(df: pd.DataFrame) -> pd.DataFrame:
    seed = 42
    np.random.seed(seed)
    random.seed(seed)

    df_zepsuty = df.copy()

    # 1. BRAKUJĄCE DANE - różne wzorce
    numeric_cols = df_zepsuty.select_dtypes(include=[np.number]).columns.tolist()
    categorical_cols = df_zepsuty.select_dtypes(include=['object', 'category']).columns.tolist()

    if len(numeric_cols) > 0:
        # MCAR - losowe braki w kolumnie numerycznej
        col_mcar = random.choice(numeric_cols)
        mask_mcar = np.random.random(len(df_zepsuty)) < 0.15
        df_zepsuty.loc[mask_mcar, col_mcar] = np.nan

        # MAR - braki zależne od innej zmiennej
        if len(numeric_cols) > 1:
            col_mar = random.choice([c for c in numeric_cols if c != col_mcar])
            threshold = df_zepsuty[col_mcar].quantile(0.7) if not df_zepsuty[col_mcar].isna().all() else 0
            mask_mar = (df_zepsuty[col_mcar] > threshold) & (np.random.random(len(df_zepsuty)) < 0.25)
            df_zepsuty.loc[mask_mar, col_mar] = np.nan

        # Kolumna z >60% braków do usunięcia
        if len(numeric_cols) > 2:
            col_duzo_brakow = random.choice([c for c in numeric_cols if c not in [col_mcar, col_mar] if 'col_mar' in locals()])
            mask_duzo = np.random.random(len(df_zepsuty)) < 0.65
            df_zepsuty.loc[mask_duzo, col_duzo_brakow] = np.nan

    if len(categorical_cols) > 0:
        # MNAR - braki w kategorycznej
        col_mnar = random.choice(categorical_cols)
        unique_vals = df_zepsuty[col_mnar].dropna().unique()
        if len(unique_vals) > 1:
            # Braki głównie w jednej kategorii
            target_val = random.choice(unique_vals)
            mask_mnar = (df_zepsuty[col_mnar] == target_val) & (np.random.random(len(df_zepsuty)) < 0.30)
            df_zepsuty.loc[mask_mnar, col_mnar] = np.nan

    # 2. DUPLIKATY
    n_duplicates = int(len(df_zepsuty) * 0.08)  # 8% duplikatów
    if n_duplicates > 0:
        duplicate_indices = np.random.choice(df_zepsuty.index, size=n_duplicates, replace=True)
        df_duplicates = df_zepsuty.loc[duplicate_indices].copy()
        df_zepsuty = pd.concat([df_zepsuty, df_duplicates], ignore_index=True)

    # 3. OUTLIERY
    if len(numeric_cols) > 0:
        for i, col in enumerate(numeric_cols[:min(3, len(numeric_cols))]):
            if df_zepsuty[col].notna().sum() > 0:
                # Różne typy outlierów
                n_outliers = int(len(df_zepsuty) * 0.03)
                outlier_indices = np.random.choice(df_zepsuty.index, size=n_outliers, replace=False)

                mean_val = df_zepsuty[col].mean()
                std_val = df_zepsuty[col].std()

                if i % 3 == 0:
                    # Ekstremalne wartości górne
                    df_zepsuty.loc[outlier_indices, col] = mean_val + (4 + np.random.random(n_outliers) * 3) * std_val
                elif i % 3 == 1:
                    # Ekstremalne wartości dolne
                    df_zepsuty.loc[outlier_indices, col] = mean_val - (4 + np.random.random(n_outliers) * 3) * std_val
                else:
                    # Mix górne i dolne
                    half = n_outliers // 2
                    df_zepsuty.loc[outlier_indices[:half], col] = mean_val + (4 + np.random.random(half) * 2) * std_val
                    df_zepsuty.loc[outlier_indices[half:], col] = mean_val - (4 + np.random.random(n_outliers-half) * 2) * std_val

    # 4. NIESPÓJNOŚCI W DANYCH
    if len(categorical_cols) > 0:
        col_niespojnosc = random.choice(categorical_cols)
        unique_vals = df_zepsuty[col_niespojnosc].dropna().unique()

        if len(unique_vals) > 0:
            # Różne warianty tej samej wartości (whitespace, case)
            n_niespojnosci = int(len(df_zepsuty) * 0.05)
            niespojnosc_indices = np.random.choice(df_zepsuty[df_zepsuty[col_niespojnosc].notna()].index,
                                                   size=min(n_niespojnosci, df_zepsuty[col_niespojnosc].notna().sum()),
                                                   replace=False)

            for idx in niespojnosc_indices:
                val = str(df_zepsuty.loc[idx, col_niespojnosc])
                # Losowy wybór problemu
                problem = random.choice(['lowercase', 'uppercase', 'whitespace', 'typo'])

                if problem == 'lowercase':
                    df_zepsuty.loc[idx, col_niespojnosc] = val.lower()
                elif problem == 'uppercase':
                    df_zepsuty.loc[idx, col_niespojnosc] = val.upper()
                elif problem == 'whitespace':
                    df_zepsuty.loc[idx, col_niespojnosc] = f"  {val}  "
                elif problem == 'typo' and len(val) > 2:
                    # Zamień losową literę
                    pos = random.randint(0, len(val)-1)
                    val_list = list(val)
                    val_list[pos] = random.choice('abcdefghijklmnopqrstuvwxyz')
                    df_zepsuty.loc[idx, col_niespojnosc] = ''.join(val_list)

    # 5. WARTOŚCI POZA SENSOWNYM ZAKRESEM
    if len(numeric_cols) > 0:
        col_nonsense = random.choice([c for c in numeric_cols if c != col_mcar])
        nonsense_indices = np.random.choice(df_zepsuty.index, size=int(len(df_zepsuty) * 0.02), replace=False)

        # Np. ujemne wartości gdzie powinny być dodatnie
        if df_zepsuty[col_nonsense].min() >= 0:
            df_zepsuty.loc[nonsense_indices, col_nonsense] = -abs(df_zepsuty.loc[nonsense_indices, col_nonsense])
        else:
            # Lub bardzo nietypowe wartości
            df_zepsuty.loc[nonsense_indices, col_nonsense] = df_zepsuty[col_nonsense].max() * 100

    # 6. PROBLEMY Z MULTIKOLINEARNOŚCIĄ - stwórz skorelowane kolumny
    if len(numeric_cols) >= 2:
        base_col = random.choice(numeric_cols)
        new_col_name = f"{base_col}_correlated"

        # Stwórz kolumnę silnie skorelowaną (r > 0.9)
        noise = np.random.normal(0, df_zepsuty[base_col].std() * 0.1, len(df_zepsuty))
        df_zepsuty[new_col_name] = df_zepsuty[base_col] * 1.5 + noise + 10

    # Wymieszaj kolejność wierszy
    df_zepsuty = df_zepsuty.sample(frac=1, random_state=seed).reset_index(drop=True)

    return df_zepsuty

In [ ]:
df_zepsuty = psuj_dataset(df_clean)
df_zepsuty

In [ ]:
from matplotlib import pyplot as plt
df_zepsuty.plot(kind='scatter', x='income', y='experience_years', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['commute_time']
  ys = series['experience_years']

  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = df_zepsuty.sort_values('commute_time', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('city')):
  _plot_series(series, series_name, i)
  fig.legend(title='city', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('commute_time')
_ = plt.ylabel('experience_years')

In [ ]:
from matplotlib import pyplot as plt
df_zepsuty.plot(kind='scatter', x='age', y='income', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
from matplotlib import pyplot as plt
df_zepsuty['experience_years'].plot(kind='hist', bins=20, title='experience_years')
plt.gca().spines[['top', 'right',]].set_visible(False)

## 4.2 Naprawianie zepsutego datasetu
Teraz można zacząć czyszczenie data frame'u

### 4.2.1a. Metoda IQR - identyfikowanie wartości odstających

In [ ]:
for col in df_zepsuty.select_dtypes(include=[np.number]).columns:
  # Obliczanie Qwantyli 1 i 3
  Q1 =  df_zepsuty[col].quantile(0.25)
  Q3 =  df_zepsuty[col].quantile(0.75)
  Q1, Q3

  # Obliczanie IQR
  IQR = Q3 - Q1
  IQR

  #Generowanie Wykresu z danych dla konkretnej kolumny

  top_line = (Q1- 1.5 * IQR)
  bottom_line = (Q3 + 1.5 * IQR)
  outliners = df_zepsuty[(df_zepsuty[col] < top_line) | (df_zepsuty[col] > bottom_line)]

  normal = df_zepsuty[(df_zepsuty[col] >= top_line) & (df_zepsuty[col] <= bottom_line)]

  plt.figure(figsize=(20, 15))

  plt.axhspan(top_line, bottom_line, alpha=0.1, color='green')

  plt.axhline(y=top_line, color='red', linestyle='--', linewidth=2,
            label=f'Dolna granica: {top_line:.2f}')
  plt.axhline(y=bottom_line, color='red', linestyle='--', linewidth=2,
            label=f'Górna granica: {bottom_line:.2f}')

  plt.scatter(normal.index, normal[col], c='blue', s=100,
            alpha=0.7, edgecolors='darkblue', linewidth=2,
            label=f'Wartości normalne ({len(normal)})', zorder=5)

  plt.scatter(outliners.index, outliners[col], c='red', s=100,
            alpha=0.7, edgecolors='darkred', linewidth=2,
            label=f'Outliery ({len(outliners)})', zorder=5)

  plt.xlabel('Indeks', fontsize=12, fontweight='bold')
  plt.ylabel(col, fontsize=12, fontweight='bold')
  plt.title('Wizualizacja metody IQR dla kolumny ' + col, fontsize=14, fontweight='bold')
  plt.grid(True, alpha=0.3, linestyle=':', linewidth=0.5)
  plt.legend(loc='best', fontsize=10)

  info = f'Q1={Q1:.2f} | Q3={Q3:.2f} | IQR={IQR:.2f} | Outliery: {len(outliners)}'
  plt.text(0.5, 0.98, info, transform=plt.gca().transAxes,
          fontsize=11, va='top', ha='center',
          bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

  plt.tight_layout()
  plt.show()

### 4.2.1.b Metoda Isolation Forest - Analizowanie wartości odstających

In [ ]:
from sklearn.ensemble import IsolationForest

# Dopasowanie modelu Isolation Forest
isolation_forest = IsolationForest(contamination=0.05)
df_zepsuty["outliers"] = isolation_forest.fit_predict(df_zepsuty[["age", "income"]])

# Wyswietlenie wartosci odstajacych
df_zepsuty[df_zepsuty["outliers"] ==-1]

In [ ]:
import plotly.express as px
fig = px.scatter(df_zepsuty, x="age", y="income", color="outliers", title="Wartosci odstajace w danych")
fig.show()

### 4.2.2. Usuwanie duplikatów

In [ ]:
df_to_clean = df_zepsuty.copy()

In [ ]:
df_to_clean = df_to_clean.drop_duplicates()

### 4.2.3. Naprawa typów danych - usuwanie stringów z kolumn numerycznych

In [ ]:
numeric_columns = df_to_clean.select_dtypes([np.number]).columns

for col in numeric_columns:
    if col in df_to_clean.columns:
        df_to_clean[col] = pd.to_numeric(df_to_clean[col], errors='coerce')

### 4.2.4. Usuwanie wartości negatywnych

In [ ]:
df_to_clean = df_to_clean[(df_to_clean[numeric_columns] >= 0).all(axis=1)]

### 4.2.5. Analiza i uzupełnianie brakujących danych

In [ ]:
plt.figure(figsize=(12, 10))
sns.heatmap(df_to_clean.isnull(), cbar=True, yticklabels=False, cmap='viridis')
plt.title('Wizualizacja brakujących danych', fontsize=14, fontweight='bold')
plt.show()

In [ ]:
for col in numeric_columns:
    if col in df_to_clean.columns:
        if df_to_clean[col].isnull().sum() > 0:
            median_value = df_to_clean[col].median()
            df_to_clean[col].fillna(median_value, inplace=True)


### 4.2.6. Resetowanie indeksów

In [ ]:
df_to_clean = df_to_clean.reset_index(drop=True)
df_to_clean

In [ ]:
if df_to_clean.columns.isin(['outliers']).any():
  df_to_clean.drop(columns=['outliers'], inplace=True)
df_to_clean

## 4.3. Detekcja wartości odstających - Isolation Forest po wstępnych poprawkach

In [ ]:
from sklearn.ensemble import IsolationForest

features_for_isolation = ['age', 'income']
X_isolation = df_to_clean[features_for_isolation].dropna()

isolation_forest = IsolationForest(contamination=0.006, random_state=42)
outliers_pred = isolation_forest.fit_predict(X_isolation)

df_to_clean['outliers'] = 1
df_to_clean.loc[X_isolation.index, 'outliers'] = outliers_pred

In [ ]:
df_to_clean[df_to_clean['outliers'] == -1]

### 4.3.1. Wizualizacja wartości odstających

In [ ]:
import plotly.express as px

fig = px.scatter(df_to_clean, x='age', y='income',
                 color='outliers',
                 color_discrete_map={1: 'blue', -1: 'red'},
                 title='Wartości odstające w danych (Isolation Forest)',
                 labels={'outliers': 'Typ', 'age': 'Wiek', 'income': 'Przychód'},
                 )
fig.update_traces(marker=dict(size=8, line=dict(width=1, color='DarkSlateGrey')))
fig.show()

## 4.4. Analiza głównych składowych (PCA)

In [ ]:
df_to_clean.columns

In [ ]:
df_to_clean = df_to_clean.drop(columns=['outliers'])

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

pca_features = ['age', 'income', 'experience_years']
X_pca = df_to_clean[pca_features].dropna()

scaler = StandardScaler()
scaled_data = scaler.fit_transform(X_pca)

pca = PCA(n_components=2)
principal_components = pca.fit_transform(scaled_data)

df_to_clean['PC1'] = np.nan
df_to_clean['PC2'] = np.nan
df_to_clean.loc[X_pca.index, 'PC1'] = principal_components[:, 0]
df_to_clean.loc[X_pca.index, 'PC2'] = principal_components[:, 1]

pca.explained_variance_ratio_

### 4.4.1. Wizualizacja komponentów głównych

In [ ]:
fig = px.scatter(df_to_clean.dropna(subset=['PC1', 'PC2']),
                 x='PC1', y='PC2',
                 color='experience_years',
                 title='Wizualizacja głównych składowych (PCA)',
                 color_continuous_scale='Viridis',
                 labels={'experience_years': 'Lata doświadczenia'})
fig.update_traces(marker=dict(size=7))
fig.show()

## 4.4a. Wizualizacja redukcji wymiarowości - t-SNE

In [ ]:
tsne_features = ['age', 'income', 'experience_years']

df_to_clean[tsne_features].corr()

In [ ]:
from sklearn.manifold import TSNE

X_tsne = df_to_clean[tsne_features].dropna()

scaler = StandardScaler()
scaled_data = scaler.fit_transform(X_tsne)

tsne = TSNE(n_components=2, random_state=42, perplexity=30, max_iter=2000, learning_rate=200)
tsne_results = tsne.fit_transform(scaled_data)

df_to_clean['tSNE1'] = np.nan
df_to_clean['tSNE2'] = np.nan
df_to_clean.loc[X_tsne.index, 'tSNE1'] = tsne_results[:, 0]
df_to_clean.loc[X_tsne.index, 'tSNE2'] = tsne_results[:, 1]

In [ ]:
df_tsne_plot = df_to_clean.dropna(subset=['tSNE1', 'tSNE2'])

plt.figure(figsize=(12, 8))
scatter = plt.scatter(df_tsne_plot['tSNE1'], df_tsne_plot['tSNE2'],
                     c=df_tsne_plot['experience_years'],
                     cmap='coolwarm',
                     s=50,
                     alpha=0.6,
                     edgecolors='black',
                     linewidth=0.5)
plt.colorbar(scatter, label='lata doświadczenia (experience_years)')
plt.xlabel('tSNE1', fontsize=12, fontweight='bold')
plt.ylabel('tSNE2', fontsize=12, fontweight='bold')
plt.title('Wizualizacja t-SNE', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4.4b. Wizualizacja redukcji wymiarowości - UMAP

In [ ]:
import umap

umap_features = ['age', 'income']
X_umap = df_to_clean[umap_features].dropna()

scaler = StandardScaler()
scaled_data = scaler.fit_transform(X_umap)

reducer = umap.UMAP(n_neighbors=10, min_dist=0.1, random_state=42)
umap_results = reducer.fit_transform(scaled_data)

df_to_clean['UMAP1'] = np.nan
df_to_clean['UMAP2'] = np.nan
df_to_clean.loc[X_umap.index, 'UMAP1'] = umap_results[:, 0]
df_to_clean.loc[X_umap.index, 'UMAP2'] = umap_results[:, 1]

In [ ]:
df_umap_plot = df_to_clean.dropna(subset=['UMAP1', 'UMAP2'])

plt.figure(figsize=(12, 8))
scatter = plt.scatter(df_umap_plot['UMAP1'], df_umap_plot['UMAP2'],
                     c=df_umap_plot['experience_years'],
                     cmap='Spectral',
                     s=50,
                     alpha=0.6,
                     edgecolors='black',
                     linewidth=0.5)
plt.colorbar(scatter, label='Lata doświadczenia (experience_years)')
plt.xlabel('UMAP1', fontsize=12, fontweight='bold')
plt.ylabel('UMAP2', fontsize=12, fontweight='bold')
plt.title('Wizualizacja UMAP', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4.5. Interaktywna analiza zależności

In [ ]:
import altair as alt

df_altair = df_to_clean.dropna(subset=['age', 'income', 'experience_years']).copy()

chart = alt.Chart(df_altair).mark_circle(size=60).encode(
    x=alt.X('age:Q', title=' (Wiek)'),
    y=alt.Y('experience_years:Q', title='Lata doświadczenia (experience_years)'),
    color=alt.Color('income:Q', scale=alt.Scale(scheme='viridis'), title='przychód ($)'),
    tooltip=['age', 'income', 'experience_years']
).properties(
    width=700,
    height=500,
    title='Interaktywna wizualizacja zależności'
).interactive()

chart

## 4.5a. Analiza macierzy korelacji

In [ ]:
correlation_features = ['age', 'income', 'experience_years']
df_corr = df_to_clean[correlation_features].dropna()
correlation_matrix = df_corr.corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm',
            center=0, fmt='.3f',
            square=True, linewidths=1,
            cbar_kws={'label': 'Współczynnik korelacji'})
plt.title('Macierz korelacji', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 4.6. Testy statystyczne - ANOVA

In [ ]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

df_to_clean['experience_category'] = pd.cut(
    df_to_clean['experience_years'],
    bins=[0, 5, 15, 50],
    labels=['Junior (0-5 lat)', 'Mid (5-15 lat)', 'Senior (15+ lat)'],
    include_lowest=True
)

df_anova = df_to_clean.dropna(subset=['income', 'experience_category']).copy()

model = ols('income ~ C(experience_category)', data=df_anova).fit()
anova_results = anova_lm(model)

p_value = anova_results['PR(>F)'][0]
f_statistic = anova_results['F'][0]

interpretation = "Różnice ISTOTNE" if p_value < 0.05 else "Różnice NIEISTOTNE"

fig, ax = plt.subplots(figsize=(12, 8))

sns.boxplot(data=df_anova, x='experience_category', y='income', palette='Set2', ax=ax)

means = df_anova.groupby('experience_category')['income'].mean()
ax.plot(range(len(means)), means, 'D', color='red', markersize=10, label='Średnia', zorder=10)

ax.set_xlabel('Kategoria doświadczenia zawodowego', fontsize=12, fontweight='bold')
ax.set_ylabel('Roczny dochód (zł)', fontsize=12, fontweight='bold')
ax.set_title('Porównanie dochodów według kategorii doświadczenia zawodowego', fontsize=14, fontweight='bold')

plt.xticks(rotation=15, ha='right')

textstr = f"Test ANOVA:\nF = {f_statistic:.4f}\np-value = {p_value:.6f}\n{interpretation}"
props = dict(boxstyle='round', facecolor='wheat', alpha=0.7)
ax.text(0.02, 0.98, textstr, transform=ax.transAxes, fontsize=11, verticalalignment='top', bbox=props)

ax.legend(loc='upper right', fontsize=10)
ax.grid(True, alpha=0.3, axis='y', linestyle=':', linewidth=0.5)

plt.tight_layout()
plt.show()